# Federated XGBoost
Several mechanisms have been proposed for training an XGBoost model in a federated learning setting.
In these examples, we illustrate the use of NVFlare to carry out *horizontal* federated learning using two approaches: histogram-based collaboration and tree-based collaboration.
And *vertical* federated learning using histogram-based collaboration.

## Horizontal Federated XGBoost
Under horizontal setting, each participant joining the federated learning will have part of 
the whole data samples / instances / records, while each sample has all the features.

### Histogram-based Collaboration
The histogram-based collaboration federated XGBoost approach leverages NVFlare integration of [federated learning support](https://github.com/dmlc/xgboost/issues/7778) in the XGBoost open-source library,
which allows the existing *distributed* XGBoost training algorithm to operate in a federated manner,
with the federated clients acting as the distinct workers in the distributed XGBoost algorithm.

In distributed XGBoost, the individual workers share and aggregate gradient information about their respective portions of the training data,
as required to optimize tree node splitting when building the successive boosted trees.

The shared information is in the form of quantile sketches of feature values as well as corresponding sample gradient and sample Hessian histograms.

Under federated histogram-based collaboration, precisely the same information is exchanged among the clients.
The main differences are that the data is partitioned across the workers according to client data ownership, rather than being arbitrarily partionable, and all communication is via an aggregating federated [gRPC](https://grpc.io) server instead of direct client-to-client communication.
Histograms from different clients, in particular, are aggregated in the server and then communicated back to the clients.

### Tree-based Collaboration
Under tree-based collaboration, individual trees are independently trained on each client's local data without aggregating the global sample gradient histogram information.
Trained trees are collected and passed to the server / other clients for aggregation and / or further boosting rounds.
Under this setting, we can further distinguish between two types of tree-based collaboration: cyclic and bagging.

#### Cyclic Training
"Cyclic XGBoost" is one way of performing tree-based federated boosting with 
multiple sites: at each round of tree boosting, instead of relying on the whole 
data statistics collected from all clients, the boosting relies on only 1 client's 
local data. The resulting tree sequence is then forwarded to the next client for 
next round's boosting. Such training scheme have been proposed in literatures [1] [2].

#### Bagging Aggregation

"Bagging XGBoost" is another way of performing tree-based federated boosting with multiple sites: at each round of tree boosting, all sites start from the same "global model", and boost a number of trees (in current example, 1 tree) based on their local data. The resulting trees are then send to server. A bagging aggregation scheme is applied to all the submitted trees to update the global model, which is further distributed to all clients for next round's boosting. 

This scheme bears certain similarity to the [Random Forest mode](https://xgboost.readthedocs.io/en/stable/tutorials/rf.html) of XGBoost, where a `num_parallel_tree` is boosted based on random row/col splits, rather than a single tree. Under federated learning setting, such split is fixed to clients rather than random and without column subsampling. 

In addition to basic uniform shrinkage setting where all clients have the same learning rate, based on our research, we enabled scaled shrinkage across clients for weighted aggregation according to each client's data size, which is shown to significantly improve the model's performance on non-uniform quantity splits.

Specifically, the global model is updated by aggregating the trees from all clients as a forest, and the global model is then broadcasted back to all clients for local prediction and further training.

The XGBoost Booster API is leveraged to create in-memory Booster objects that persist across rounds to cache predictions from trees added in previous rounds and retain other data structures needed for training.

## Vertical Federated XGBoost
Under vertical setting, each participant joining the federated learning will 
have part of the whole features, while each site has all the overlapping instances.

### Private Set Intersection (PSI)
In this tutorial, we assume that all parties hold the same population but different features. 

In reality, however, not every site will have the same set of data samples (rows), ad we shall use PSI to first compare encrypted versions of the sites' datasets in order to jointly compute the intersection based on common IDs. To learn more about our PSI protocol implementation, see our [psi example](https://github.com/NVIDIA/NVFlare/tree/main/examples/advanced/psi/README.md).

### Histogram-based Collaboration
Similar to its horizontal counterpart, under vertical collaboration, histogram-based collaboration will 
aggregate the gradient information from each site and update the global model accordingly, resulting in
the same model as the centralized / histogram-based horizontal training. 
We leverage the [vertical federated learning support](https://github.com/dmlc/xgboost/issues/8424) in the XGBoost open-source library. This allows for the distributed XGBoost algorithm to operate in a federated manner on vertically split data.

## Data Preparation
Download and Store Data
To run the examples, we first download the dataset from this link, which is a single .csv file. By default, we assume the dataset is downloaded, uncompressed, and stored in /tmp/nvflare/dataset/creditcard.csv. Alternatively, the following can be used to download the data

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("mlg-ulb/creditcardfraud")

print("Path to dataset files:", path)

### Data Split
To prepare data for further experiments, we perform the following steps:
1. Split the dataset into training/validation and testing sets. 
2. Split the training/validation set: 
    * Into "train" and "valid" for baseline centralized training.
    * Into "train" and "valid" for each client under horizontal setting. 
    * Into "train" and "valid" for each client under vertical setting.

Data splits used in this example can be generated with

In [ ]:
%%sh 
prepare_data.sh

This will generate data splits for 3 clients under all experimental settings.

> **_NOTE:_** In this section, we have divided the dataset into separate columns for each site,
> assuming that the datasets from different sites have already been joined using Private Set
> Intersection (PSI). However, in practice, each site initially has its own separate dataset. To
> combine these datasets accurately, you need to use PSI to match records with the same ID across
> different sites. 

> **_NOTE:_** The generated data files will be stored in the folder `/tmp/nvflare/dataset/xgb_dataset/`,
> and will be used by jobs by specifying the path within `config_fed_client`

## Experiments
We first run the centralized trainings to get the baseline performance, then run the federated XGBoost training using NVFlare Simulator via [JobAPI](https://nvflare.readthedocs.io/en/main/programming_guide/fed_job_api.html).

### Centralized Baselines
For centralize training, we train the XGBoost model on the whole dataset, as well as subsets with different subsample rates
and parallel tree settings.

In [ ]:
%%sh 
run_experiment_centralized.sh ${DATASET_ROOT}

The results by default will be stored in the folder `/tmp/nvflare/workspace/xgboost/centralized/`.

### Horizontal Experiments
The following cases will be covered:
- Histogram-based collaboration
- Tree-based collaboration with cyclic training 
- Tree-based collaboration with bagging training 

The experiments can be run with:

In [ ]:
%%sh
run_experiment_horizontal_histogram.sh
run_experiment_horizontal_tree.sh
run_experiment_vertical.sh